In [1]:
#!/usr/bin/env python3

# setup lsst_distrib
# ds9 &

# imports
import os
import sys
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy.stats as stats
from collections import Counter as count
import pandas as pd

# LSST stack imports
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.detection as afwDetection
import lsst.afw.image as afwImage

# LSK imports
from lskfunctions import *

# # interactive plotting setup
# matplotlib.use('Qt5Agg')
# plt.ion()

In [2]:
# setup
visit = 1252
datadir = '/datasets/hsc/repo/rerun/private/erykoff/hscproc/deltaoverscan/run5_vanilla'
outdir = '/project/lskelvin/amp2amp/amppiston'
butler = dafPersist.Butler(datadir)
camera = butler.get('camera', visit=int(visit))
visdir = outdir + '/v' + f'{visit:07}'
outreadme = visdir + '/README.md'
if not os.path.exists(visdir):
    os.makedirs(visdir)
if os.path.exists(outreadme):
    os.remove(outreadme)

In [3]:
# loop over each CCD
readme = '# Visit ' + str(visit) + '\n'
for i in range(len(camera)):
    # progress
    update_progress("Visit %s" % visit, i / (len(camera) + 1))
    outfile = visdir + '/v' + f'{visit:07}' + f'c{i:03}' + '.png'
    # get data
    try:
        ccd = butler.get('postISRCCD', visit=int(visit), ccd=i)
    except:
        continue
    # masked data
    good = (((ccd.mask.array & ccd.mask.getPlaneBitMask("BAD")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("SAT")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("INTRP")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("CR")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("EDGE")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("DETECTED")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("DETECTED_NEGATIVE")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("SUSPECT")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("NO_DATA")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("CROSSTALK")) == 0)
           & ((ccd.mask.array & ccd.mask.getPlaneBitMask("UNMASKEDNAN")) == 0))
    ccdmasked = ccd.image.array*good
    ccdmasked[ccdmasked==0] = np.nan
    # clipping limits
    med = np.median(stats.sigma_clip(ccdmasked.flatten()[~np.isnan(ccdmasked.flatten())], sigma=3, masked=False))
    # amps
    amp1 = ccdmasked[:,0:512] - med
    amp2 = ccdmasked[:,512:1024] - med
    amp3 = ccdmasked[:,1024:1536] - med
    amp4 = ccdmasked[:,1536:2048] - med
    # pistons
    amp1vals = amp1.flatten()[~np.isnan(amp1.flatten())]
    amp2vals = amp2.flatten()[~np.isnan(amp2.flatten())]
    amp3vals = amp3.flatten()[~np.isnan(amp3.flatten())]
    amp4vals = amp4.flatten()[~np.isnan(amp4.flatten())]
    if len(amp1vals) > 0:
        ped1 = np.median(stats.sigma_clip(amp1vals, sigma=3, masked=False))
    else:
        ped1 = 0
    if len(amp2vals) > 0:
        ped2 = np.median(stats.sigma_clip(amp2vals, sigma=3, masked=False))
    else:
        ped2 = 0
    if len(amp3vals) > 0:
        ped3 = np.median(stats.sigma_clip(amp3vals, sigma=3, masked=False))
    else:
        ped3 = 0
    if len(amp4vals) > 0:
        ped4 = np.median(stats.sigma_clip(amp4vals, sigma=3, masked=False))
    else:
        ped4 = 0
    pis1 = np.full(amp1.shape, ped1)
    pis2 = np.full(amp2.shape, ped2)
    pis3 = np.full(amp3.shape, ped3)
    pis4 = np.full(amp4.shape, ped4)
    pistons = np.concatenate((pis1, pis2, pis3, pis4), axis=1)
    # generate image
    fontsize = 8
    fig = plt.figure(figsize=(5,3), dpi=300)
    gs = fig.add_gridspec(1,7)
    ax1 = fig.add_subplot(gs[0, 0:2], frameon=True)
    ax2 = fig.add_subplot(gs[0, 2:4], frameon=True)
    ax3 = fig.add_subplot(gs[0, 4:6], frameon=True)
    ax4 = fig.add_subplot(gs[0, 5:], frameon=True)
    ax1.axis('off'); ax1.set_title('CCD %s, vanilla' % i, fontsize=fontsize)
    ax2.axis('off'); ax2.set_title('CCD %s, with pistons' % i, fontsize=fontsize)
    ax3.axis('off'); ax3.set_title('pistons', fontsize=fontsize)
    ax4.axis('off')
    _,zlo,zhi = imprep(ccdmasked, scaletype='asinh', scalemode=99, colmap='sls', smoothfwhm=3)
    orig,*_ = imprep(ccd.image.array, scaletype='asinh', zlo=zlo[0], zhi=zhi[0], colmap='sls', smoothfwhm=3)
    rep,*_ = imprep(ccd.image.array-pistons, scaletype='asinh', zlo=zlo[0], zhi=zhi[0], colmap='sls', smoothfwhm=3)
    ax1.imshow(orig, origin='lower')
    ax2.imshow(rep, origin='lower')
    cbim = ax3.imshow(pistons, cmap=plt.cm.get_cmap('RdYlBu', 13), vmin=-1.25, vmax=1.25, origin='lower')
    ax3.text(256, 256, round(ped1, ndigits=2), fontsize=5, ha='center', va='center')
    ax3.text(256+512, 256, round(ped2, ndigits=2), fontsize=5, ha='center', va='center')
    ax3.text(256+1024, 256, round(ped3, ndigits=2), fontsize=5, ha='center', va='center')
    ax3.text(256+1536, 256, round(ped4, ndigits=2), fontsize=5, ha='center', va='center')
    cb = fig.colorbar(cbim, ax=ax4, aspect=10)
    cb.ax.tick_params(labelsize = 8)
    plt.gca().set_axis_off()
    plt.subplots_adjust(left = 0.02, right = 0.9, bottom = 0.01, top = 0.95, hspace = 0, wspace = 0.1)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.savefig(outfile)
    plt.close()
    # readme
    readme = readme + '\n### CCD ' + str(ccd) + '\n![](' + os.path.basename(outfile) + ')'

# finish up
update_progress("Visit %s" % visit, 1)
text_file = open(outreadme, "w")
n = text_file.write(readme)
text_file.close()

Visit 1252: [############################################################] 100% DONE
